# Day 22

In [1]:
with open('data/day22.txt') as fn:
    instructions = fn.read().split('\n')

In [2]:
def cut(cards, cut):
    return cards[cut:] + cards[:cut]

In [3]:
def deal_into_stack(cards):
    return cards[::-1]

In [4]:
def deal_increment(cards, n):
    out = cards.copy()
    i = 0
    for c in cards:
        out[i] = c
        i = (i+n) % len(cards)
    return out
        

In [5]:
def parse_instruction(instruction, cards):
    parts = instruction.split(" ")
    if parts[0] == 'cut':
        return cut(cards, int(parts[-1]))
    
    if (parts[0]=='deal') & (parts[1]=='with'):
        return deal_increment(cards, int(parts[-1]))
    
    if (parts[0]=='deal') & (parts[1]=='into'):
        return deal_into_stack(cards)
    

In [6]:
cards = [i for i in range(10_007)]

In [7]:
for inst in instructions:
    cards = parse_instruction(inst, cards)

In [8]:
cards.index(2019)

3036

### We can do this for 1 card

In [9]:
def cut_p(pos, n_card, cut):
    return (pos - cut )% n_card

In [10]:
def deal_into_stack_p(pos, n_card):
    return n_card - pos -1

In [11]:
def deal_increment_p(pos, n_card, n):
    return (pos * n) % n_card       

These can just be composed:

In [12]:
pos = 2020
n_card = 10_007

pos = cut_p(pos, n_card, 500)
pos = deal_into_stack_p(pos, n_card)
pos = deal_increment_p(pos, n_card, 25)

In [13]:
pos

2003

All transformations can be composed to the form:

` ax+b % n_card`

First year number theory time

In [14]:
def cut_t(a, b, cut):
    return a, b-cut
def deal_into_t(a, b):
    return -a, -b-1
def deal_increment_t(a, b, n):
    return n*a, n*b

In [15]:
a = 1
b = 0

pos=2020
n_card = 10_007

a, b = cut_t(a, b, 500)
a, b = deal_into_t(a, b)
a, b = deal_increment_t(a, b, 25)

a = a % n_card
b = b % n_card

(a * pos + b) % n_card

2003

In [16]:
def parse_instruction_t(instruction, a, b):
    parts = instruction.split(" ")
    if parts[0] == 'cut':
        return cut_t(a, b, int(parts[-1]))
    
    if (parts[0]=='deal') & (parts[1]=='with'):
        return deal_increment_t(a, b, int(parts[-1]))
    
    if (parts[0]=='deal') & (parts[1]=='into'):
        return deal_into_t(a, b)
    

In [17]:
a = 1
b = 0
for inst in instructions:
    a, b = parse_instruction_t(inst, a, b)
    a = a % n_card
    b = b % n_card

In [18]:
a, b

(6833, 6862)

In [19]:
pos = 2019
(a * pos + b) % n_card

3036

# Q2

In [20]:
times = 101741582076661
n_card = 119315717514047

In [21]:
a = 1
b = 0
for inst in instructions:
    a, b = parse_instruction_t(inst, a, b)
    a = a % n_card
    b = b % n_card

In [22]:
a, b

(85946586836261, 89183823480133)

Is a coprime to n_card? - We can find inverse if so

In [23]:
# from google/stackoverflow
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def coprime(a, b):
    return gcd(a, b) == 1

In [24]:
coprime(a,n_card)

True

We can use a-1

In [25]:
# From google/stackoverflow
def modInverse(a, m) : 
    m0 = m 
    y = 0
    x = 1
  
    if (m == 1) : 
        return 0
  
    while (a > 1) : 
  
        # q is quotient 
        q = a // m 
  
        t = m 
  
        # m is remainder now, process 
        # same as Euclid's algo 
        m = a % m 
        a = t 
        t = y 
  
        # Update x and y 
        y = x - q * y 
        x = t 
  
  
    # Make x positive 
    if (x < 0) : 
        x = x + m0 
  
    return x 

In [26]:
a_inv = modInverse(a, n_card) % n_card
b_inv = -(a_inv * b) % n_card

### Test the inverse

In [27]:
(a * 2020 + b) % n_card

96920249788968

In [28]:
(a_inv * 96920249788968 + b_inv) % n_card

2020

Do it 101741582076661 times

We can series expand the powers

$$f(x) = ax + b$$

$$f^{n}(x) = a^n x + \frac{b(1-a^n)}{1-a}$$

In [29]:
a_final = pow(a_inv, times, n_card)
b_final = b_inv*(((pow(a_inv, times, n_card)-1)) * modInverse((a_inv-1), n_card)) % n_card

In [30]:
(a_final * 2020 + b_final) % n_card

70618172909245